#[Work work, money money][repeatyoutube]
Combine job board and BLS data to find trends in job / industry growth in Chicago and elsewhere.

##Data:
- BLS: http://www.bls.gov/help/hlpforma.htm,
    + Occupational Employment Statistics http://download.bls.gov/pub/time.series/oe/
    + National Compensation Survey http://download.bls.gov/pub/time.series/nc/
- Census:
    + county shapefiles https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html
    + or, better, Mike Bostock's [US-atlas project] [usatlas] (census shapefiles ⇒ GeoJSON)
- [JobsAggregator][ja]: (results from all 5 of [Indeed][indeed], [SimplyHired][simply], [CareerBuilder][cb], [Monster][monster], and [CareerJet][cj]): 

##Technology:
- Apache Spark on an Amazon EC2 (Elastic Cloud 2) cluster; instructions below
- input data stored in Amazon S3 buckets, output written to HDFS permanent storage
- Images rendered in D3 via a private [Lightning][lightning] server

[bls_api]: http://www.bls.gov/developers/api_python.htm
[cb]: http://developer.careerbuilder.com/
[cj]: http://www.careerjet.com/partners/
[indeed]: http://www.indeed.com/publisher
[ja]: http://www.jobsaggregator.com/US/
[lightning]: http://lightning-viz.org/documentation/
[monster]: http://partner.monster.com/developers
[repeatyoutube]: http://listenonrepeat.com/watch/?v=SvpsoEOJ0_E
[simply]: https://simply-partner.com/partners-signup
[usatlas]: https://github.com/mbostock/us-atlas